In [ ]:
!pip install opencv-contrib-python
!pip install moms_apriltag
!pip install imutils

In [3]:
import numpy as np
from moms_apriltag import TagGenerator2
import cv2
import imutils

import argparse
import sys
from tqdm import tqdm
import os

### Preparing ArUco & AprilTag markers

Generating ArUco markers is a pretty simple process which require the same version of ```opencv-contrib-python``` that is used with AMPy. For AprilTag, you will need to install the ```moms_apriltag``` and ```imutils``` packages.

In [5]:
marker_source = 'april' 
# marker_source = 'aruco'

In [4]:
ARUCO_DICT = {
	"DICT_4X4_50": cv2.aruco.DICT_4X4_50,
	"DICT_4X4_100": cv2.aruco.DICT_4X4_100,
	"DICT_4X4_250": cv2.aruco.DICT_4X4_250,
	"DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
	"DICT_5X5_50": cv2.aruco.DICT_5X5_50,
	"DICT_5X5_100": cv2.aruco.DICT_5X5_100,
	"DICT_5X5_250": cv2.aruco.DICT_5X5_250,
	"DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
	"DICT_6X6_50": cv2.aruco.DICT_6X6_50,
	"DICT_6X6_100": cv2.aruco.DICT_6X6_100,
	"DICT_6X6_250": cv2.aruco.DICT_6X6_250,
	"DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
	"DICT_7X7_50": cv2.aruco.DICT_7X7_50,
	"DICT_7X7_100": cv2.aruco.DICT_7X7_100,
	"DICT_7X7_250": cv2.aruco.DICT_7X7_250,
	"DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
	"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,
	"DICT_APRILTAG_16h5": cv2.aruco.DICT_APRILTAG_16h5,
	"DICT_APRILTAG_25h9": cv2.aruco.DICT_APRILTAG_25h9,
	"DICT_APRILTAG_36h10": cv2.aruco.DICT_APRILTAG_36h10,
	"DICT_APRILTAG_36h11": cv2.aruco.DICT_APRILTAG_36h11
}
tg = TagGenerator2('tag36h11')

# general parameters
output_dir = "markers/"
serial_literal = "A"
markers_number = 100

# marker properties
marker_type = "DICT_7X7_1000" # for borders you can specify another type to decrease error chance
top_margin, bottom_margin = 50, 50
dsize = (350, 473) # final size of the exported marker
font = cv2.FONT_HERSHEY_DUPLEX
fontScale = 1.5

# list with processed tags applied in further manipulations
markers_aruco = []

try:
    os.mkdir(marker_source + '_' + output_dir)
except:
    pass

for marker_id in tqdm(range(markers_number)):
    
    # create a marker
    arucoDict = cv2.aruco.Dictionary_get(ARUCO_DICT[marker_type])
    
    if marker_source== 'aruco':
        tag = np.zeros((300, 300, 1), dtype="uint8")
        cv2.aruco.drawMarker(arucoDict, marker_id, 300, tag, 1)
        print(tag.shape)
    if marker_source == 'april':
        tag = tg.generate(marker_id)
        tag = imutils.resize(tag, width=300)[:, :, None]
        
    

    # add top space for a text
    for i in range(top_margin):
        tag = np.r_[np.array([[[255] for i in range(300)]]).T, tag]

    # add bottom space (optional)
    for i in range(bottom_margin):
        tag = np.r_[tag, np.array([[[255] for i in range(300)]]).T]

    # left border line 
    for i in range(3):
        tag = np.r_[np.array([[[0] for i in range(300)]]).T, tag]
    
    # right border line
    for i in range(3):
        tag = np.r_[tag, np.array([[[0] for i in range(300)]]).T]    
    
    # top border line
    for i in range(3):
        for j in range(len(tag)):
            tag[j][i][0] = 0

    # resize the marker to fit a printing page
    tag = cv2.resize(tag.astype('float32'), dsize)
    
    # insert a label in the center of the top space
    text = serial_literal + str(marker_id)
    textsize = cv2.getTextSize(text, font, fontScale, 2)[0]
    textX = (tag.shape[1] - textsize[0]) / 2
    org, color, thickness = (int(textX) , 46), (0, 0, 0), 3
    image = cv2.putText(cv2.UMat(tag), text, org, font,fontScale, color, thickness, cv2.LINE_AA)
    
    markers_aruco.append(image)
    
    cv2.imwrite(marker_source + '_' + output_dir + serial_literal + str(marker_id) + ".jpg", image)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 30.76it/s]
